# Ethanol Water Distillation

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import math
import ipywidgets as widgets
from IPython.display import display

In [6]:
def rectifying_x_to_y(R, xD, x):
    slope = (R / (R+1))
    yint = (xD / (R+1))
    return (slope * x + yint)

def rectifying_y_to_x(R, xD, y):
    slope = ((R+1) / R)
    yint = -(xD / R)
    return (slope * y + yint)
    

In [24]:
x_eth_mol = [0, 1.9,  7.21,  9.66, 12.38, 16.61, 23.37, 26.08, 32.73, 39.65, 50.79, 51.98, 57.32, 67.63, 74.72, 89.43]
y_eth_mol = [0,  17, 38.91, 43.75, 47.04, 50.89, 54.45,  55.8, 58.26, 61.22, 65.64, 65.99, 68.41, 73.85, 78.15, 89.43]
x_eth_wt = []
y_eth_wt = []
x_axis = np.linspace(0,1,100)
for i in range(len(x_eth_mol)):
    x_eth_wt.append(x_eth_mol[i]*46.07 / (x_eth_mol[i]*46.07 + (100-x_eth_mol[i])*18.02))
    y_eth_wt.append(y_eth_mol[i]*46.07 / (y_eth_mol[i]*46.07 + (100-y_eth_mol[i])*18.02))
    
yx_fit = np.poly1d(np.polyfit(x_eth_wt, y_eth_wt, 8))


yx_fit_vals = []
for i in range(len(x_axis)):
    if (yx_fit(x_axis[i]) >= x_axis[i]):
        yx_fit_vals.append(yx_fit(x_axis[i]))
    else:
        yx_fit_vals.append(x_axis[i])


def distillation_plot(xD, R):
    plt.figure(figsize=(6, 6))
    plt.plot(x_axis, yx_fit_vals, label = 'Equilibrium Curve')
    plt.plot(x_axis, x_axis, 'k--', label = '45° Line')
    plt.scatter(x_eth_wt, y_eth_wt, s=25, alpha=0.5)
    xF = 0.075
    plt.plot([xF, xF],[xF, yx_fit(xF)], label = 'q-line Sat Liq')
    
    rect = []
    op_color = 'green'
    for i in range(int(xF*100), int(xD*100 +1)):
        rect.append(rectifying_x_to_y(R, xD, x_axis[i]))
        if (rectifying_x_to_y(R, xD, x_axis[i]) >= yx_fit(x_axis[i])):
            op_color = 'red'
    plt.plot(x_axis[int(xF*100): int(xD*100 +1)], rect, label = 'Operating Line', color = op_color)
    
    #MCCABE-THIELE ALGORITHM USING THE OPERATING LINES
    N = 0
    x1, x2, y1, y2 = xF, xF, yx_fit(xF), yx_fit(xF)
    while (x1 < xD) and (rectifying_y_to_x(R, xD, y1) > x1):
        N += 1
        x2 = rectifying_y_to_x(R, xD, y1)
        #print(x2)
        plt.plot([x1, x2],[y1, y1], 'k--', alpha=0.3)
        y2 = yx_fit(x2)
        plt.plot([x2, x2],[y1, y2], 'k--', alpha=0.3)
        x1 = x2
        y1 = y2
        #print(x1, x2, y1, y2)
    
    if op_color == 'green':
        print("There are {} Theoretical stages".format(N))
    

    plt.xlabel('$x_{EtOH}$', fontsize = 16)
    plt.ylabel('$y_{EtOH}$', fontsize = 16)
    plt.legend()
    plt.xlim(0,1)
    plt.ylim(0,1)

    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()

xD_slider = widgets.FloatSlider(value = 0.95, min=0, max=1,      step=0.01, description='$x_D$:',        readout_format='.2f')
R_slider =  widgets.FloatLogSlider(value=1.0, base=10, min=-2, max=3, step=0.1,  description='Reflux Ratio:', readout_format='.1f')

# Create the interactive plot
interactive_plot = widgets.interactive(distillation_plot, xD=xD_slider, R = R_slider)

display(interactive_plot)

interactive(children=(FloatSlider(value=0.95, description='$x_D$:', max=1.0, step=0.01), FloatLogSlider(value=…